In [1]:
%pip install pandas
%pip install numpy
%pip install scipy
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import pandas for data manipulation
import pandas as pd
# Import TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
# Import cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\berai\anaconda3\ipykernel_29112\4278529357.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## EDA

We will be running simple recommendation system based on IMDB description similarity for NLP

In [3]:
import pandas as pd

IMDB_PATH="data/normalized_tables/imdb.csv"

df=pd.read_csv(IMDB_PATH)
df.head()

,kdrama_name,imdb_rating,imdb_users,imdb_description
0,100 Days My Prince,100.0,NaN,"Upon losing his memory, a crown prince encount..."
1,12 Signs of Love,12.0,NaN,NaN
2,12 Years Promise,12.0,190.0,A pregnant teen is forced by her family to lea...
3,18 Again,18.0,NaN,A 37-year-old man on the verge of being divorc...
4,1st Republic,1.0,NaN,NaN


In [4]:
count=len(df)
na_count=df.isna().sum()
count

1471

In [5]:
na_count

kdrama_name           0
imdb_rating         614
imdb_users          892
imdb_description    664
dtype: int64

In [6]:
tfidf = TfidfVectorizer(analyzer='word',
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), 
                      stop_words = 'english')

# Filling NaNs with empty string
df['imdb_description'] = df['imdb_description'].fillna('')

In [23]:
# Fitting the TF-IDF on the 'overview' text
tfidf_matrix = tfidf.fit_transform(df['imdb_description'])

tfidf_matrix.shape

(1471, 44870)

In [24]:
# Compute the Cosine Similarity
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Create a pandas series with movie titles as indices and indices as series values 
indices = pd.Series(df.index, index=df['kdrama_name']).drop_duplicates()

### Testing out similarities

- we will choose random movie and find top  5 movies similar to it

In [25]:
chosen_movie=df.sample()
chosen_movie

,kdrama_name,imdb_rating,imdb_users,imdb_description
192,Children of Nobody,7.8,290.0,"Cha Woo Kyung is a child counselor, happily ma..."


In [58]:
title=chosen_movie["kdrama_name"]

# Get the index corresponding to movie title
index = indices[title]

# Get the cosine similarity scores 
similarity_scores = list(enumerate(similarity_matrix[index][0]))

# Sort the similarity scores in descending order
sorted_similarity_scores = sorted(similarity_scores,  key=lambda x: x[1], reverse=True)

# Top-10 most similar movie scores
top_10_movies_scores = sorted_similarity_scores[1:11]

# Get movie indices
top_10_movie_indices=[]
for i in top_10_movies_scores:
    top_10_movie_indices.append(i[0])
    
df['kdrama_name'].iloc[top_10_movie_indices]

1157        Tell Me What You Saw
121                  Beyond Evil
868                 Nobody Knows
1349                      Tunnel
1341                       Train
951             Psychopath Diary
1054                  Seoul 1945
1411    When the Camellia Blooms
1173                  The Chaser
1200         The Good Bad Mother
Name: kdrama_name, dtype: object